In [1]:
import MetaTrader5 as mt5
import numpy as np
import requests
from bs4 import BeautifulSoup
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import joblib
import matplotlib.pyplot as plt



In [2]:
import keras
import tensorflow as tf
print("Keras version:", keras.__version__)
print("TensorFlow version:", tf.__version__)

Keras version: 2.10.0
TensorFlow version: 2.10.1


In [3]:
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

mt5.initialize()

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4200


True

In [4]:
if not mt5.initialize():
    print('Initialization failed, check internet connection. You must have Meta Trader 5 installed.')
    mt5.shutdown()

else:
    print(mt5.account_info()._asdict())
    print("\n")
    print(mt5.terminal_info()._asdict())
    print("\n")
    print(mt5.symbols_total())



{'login': 213644473, 'trade_mode': 0, 'leverage': 1000, 'limit_orders': 500, 'margin_so_mode': 0, 'trade_allowed': True, 'trade_expert': True, 'margin_mode': 2, 'currency_digits': 2, 'fifo_close': False, 'balance': 200.0, 'credit': 0.0, 'profit': 0.0, 'equity': 200.0, 'margin': 0.0, 'margin_free': 200.0, 'margin_level': 0.0, 'margin_so_call': 25.0, 'margin_so_so': 15.0, 'margin_initial': 0.0, 'margin_maintenance': 0.0, 'assets': 0.0, 'liabilities': 0.0, 'commission_blocked': 0.0, 'name': 'micheal bright omage ', 'server': 'OctaFX-Demo', 'currency': 'USD', 'company': 'Octa Markets Incorporated'}


{'community_account': False, 'community_connection': False, 'connected': True, 'dlls_allowed': False, 'trade_allowed': True, 'tradeapi_disabled': False, 'email_enabled': False, 'ftp_enabled': False, 'notifications_enabled': False, 'mqid': False, 'build': 5200, 'maxbars': 100000, 'codepage': 0, 'ping_last': 216118, 'community_balance': 0.0, 'retransmission': 0.0, 'company': 'MetaQuotes Software

In [5]:
account = mt5.account_info()
terminal = mt5.terminal_info()

print(account.equity)

if(terminal.connected == True and terminal.trade_allowed == True):
    print("AI is successfully functional")
else:
    print("Please make sure metatrade 5 has internet and algo Trade is Turn On")

200.0
AI is successfully functional


In [6]:
symbols = mt5.symbols_get()
print(len(symbols))
t_symbol = ["GBPUSD","EURUSD","USDCAD","USDJPY","AUDUSD"]
for symbol in symbols:
    for t_s in t_symbol:
        if(symbol.name == t_s):
            print("yes ",t_s)
            break
       
        

280
yes  EURUSD
yes  GBPUSD
yes  USDJPY
yes  AUDUSD
yes  USDCAD


In [7]:
markets = []
for t_s in t_symbol:
    market = mt5.copy_rates_from_pos(t_s, mt5.TIMEFRAME_M30, 0, 99999)  
    print(market.shape)
    markets.append(market)

len(markets)

(99999,)
(99999,)
(99999,)
(99999,)
(99999,)


5

In [8]:
market

array([(1502859600, 0.78367, 0.78374, 0.78309, 0.78323, 1585,  7, 0),
       (1502861400, 0.78324, 0.78337, 0.78255, 0.78298, 1354,  7, 0),
       (1502863200, 0.78296, 0.78338, 0.78285, 0.78329, 1115,  7, 0), ...,
       (1756360800, 0.6519 , 0.65213, 0.65171, 0.65176, 1104, 14, 0),
       (1756362600, 0.65176, 0.65185, 0.65123, 0.65138, 1015, 14, 0),
       (1756364400, 0.65138, 0.65139, 0.65106, 0.6511 ,  302, 14, 0)],
      dtype=[('time', '<i8'), ('open', '<f8'), ('high', '<f8'), ('low', '<f8'), ('close', '<f8'), ('tick_volume', '<u8'), ('spread', '<i4'), ('real_volume', '<u8')])